#Assignment 2 - Transliteration

In this task you are required to solve the transliteration problem of names from English to Russian. Transliteration of a string means writing this string using the alphabet of another language with the preservation of pronunciation, although not always.


## 0. Instructions

To complete the assignment please do the following  steps (all the three are requred to get the full credits): 

1. **Notebook**. Upload to Canvas a filled notebook with answers (this file). 
Please enter the questions inside this notebook where appropriate in the designated cells.

2. **Scripts**. In *Practical* part of this notebook you will be asked to implement a transformer-based approach for transliteration. Upload to Canvas your code which implements solution to this part in the form of *.py file (not *.ipynb notebooks) of the model:

  - ``translit.py`` 

This script should have the specific structure as it is shown in the baseline soultion [here](https://github.com/skoltech-nlp/filimdb_evaluation/blob/master/translit_baseline.py). So, you should implement your ``train`` and ``classify`` functions. Your model should be implemented as a special class/function in this script (be sure if you add any outer dependencies that everything is improted correctly and can be reproducable). 

Please make sure that you code does not contain any dependencies which are specific to your system.

  *Important*: to make sure everything works, please use ONLY the following software configuration (no matter which operating system you use): Anaconda 2020.07  distribution for Python 3.8 and PyTorch 1.3. The preferred way to install PyTorch is "conda install -c pytorch pytorch" and Torchtext is "conda install -c pytorch torchtext". There should be no additional libraries used: Anaconda already provides a sufficient number of them. If you need something just select from these available. Test for no the absence of dependencies by creating a virtual environment. 

3. **Shared task.** After the implementation of models' architectures you are asked to participate in the [competition](https://competitions.codalab.org/competitions/30932) to solve **Transliteration** task using your implemented code. 

You should use your transliteration scripts from the previous part to train, validate, and generate predictions for the public (Practice) and private (Evaluation) test sets. For this you should use [``evaluate_translit.py``](https://github.com/skoltech-nlp/filimdb_evaluation/blob/master/evaluate_translit.py) script. 

It will produce predictions (``preds_translit.tsv``) for the dataset and score them if the true answers are present. You can use these scores to evaluate your model on dev set and choose the best one. Be sure:

1. To download the [dataset](https://github.com/skoltech-nlp/filimdb_evaluation/blob/master/TRANSLIT.tar.gz) and unzip it in the same folder where ``evaluate_translit.py`` is.
2. to put your ``translit.py`` and ``evaluate_translit.py`` scripts in the same folder. 
3. to change importing in ``evaluate_translit.py`` from ``translit_baseline`` import ``train, classify`` to ``from translit import train, classify``.

If you work in Colab you can dowload data and scripts with ``wget`` command and run them from notebook cells. 

Upload obtained TSV file with your predictions (``preds_translit.tsv``) in ``.zip`` for the best results to both phases of the competition.

Please, provide here in the notebook your user name in Codalab competition that we can recognize you in the leaderboard.


**YOUR USERNAME IN THE CODALAB LEADERBOARD:**

```

ENTER HERE

```

##1. Theoretical part
Solve the following theoretical subtasks:


###1.1 Normalization

* The main part of transformer architecture is MultiHeadAttention. This layer is breafly explained in subsection 2.2, its logic is described with formulas:
$$
Attention(Q, K, V)=softmax\left(\frac{Q K^{T}}{\sqrt{d_{k}}}\right) V
$$
$$
MultiHead(Q, K, V) = Concat\left(head_1, ..., head_h\right) W^O \\
$$
$$
head_i = Attention \left(Q W_i^Q, K W_i^K, V W_i^V\right)
$$
$h$ - the number of attention heads - parallel sub-layers for Scaled Dot-Product Attention on a vector of smaller dimension ($d_{k} = d_{q} = d_{v} = d_{model} / h$). 
Explain your thoughts: what the reason of dividing before $softmax$ the elements of $QK^{T}$ by $\sqrt{d_{k}}$\,?
($d_k = d_{model} / h$ - dimentionality of vectors-keys, queries and values, $d_{model}$ - dimentionality of inout and output vectors at all layers, $h$ - number of attention heads).
What problems can arise making training less stable without devision by $\sqrt{d_{k}}$ and working with large $d_{model}$?


For large values of $d_{k}$, the dot products grow large in magnitude, pushing the softmax function into regions where it has
extremely small gradients. To deal with this effect, the dot products are scaled by $1/d_{k}$.

###1.2 Complexity



* Let the input of Transformer be the sequence of $n$ tokens. Before it is processed by MultiHeadAttention layer (MHA) it corresponds to the sequence of vectors $x_{1} \ldots x_{n}$. Output from this layer -- if the sequence of vectors $y_{1} \ldots y_{n}$, vectors can be packed in a matrix $Y$ and be denoted with formula $Y = \mathbf{MHA}(x_{1} \ldots x_{n})$.
Imagine now that there is a hidden layer of a recurrent neural network that performs a transformation similar to the type of input and output $y_{i} = \mathbf{RecurrentLayer}(x_{1} \ldots x_{i})$.
Let us introduce the function $path(i,j)$, which denotes the number of applications of various operations on the path of creating a vector $y_{j}$ from a sequence $x_{i} \ldots x_{j}$. In terms of O-big $O(\cdot)$, dependent from  $n$ describe, what equals $path(1,n)$ for recurrent layer and  MultiHeadAttention layer. 
e.x. $O(n\log(n)),O(n^{2}),O(1)$



Reccurent: O(n),
MultiHeadAttention: O(1)

###1.3 Mask


The mask is applied to the weights of MultiHeadAttention in transformer before computing the weighted sum of value-vectors. Describe what types of masks are usually used.

#### Padding mask: 
The masked steps are at the start or the end of a sequence. Padding comes from the need to encode sequence data into contiguous batches: in order to make all sequences in a batch fit a given standard length, it is necessary to pad or truncate some sequences.
#### Look-ahead mask:
It can make the neural network ignore the future values on the decoder input (the target input).


### 1.4 Positional encoding 

As you remember, Transformer treats an input sequence of elements as a time series. Since the Encoder inside the Transformer simultaneously processes the entire input sequence, the information about the position of the element needs to be encoded inside its embedding, since it is not identified in any other way inside the model. That is why the PositionalEncoding layer is used, which sums embeddings with a vector of the same dimension.
Let the matrix of these vectors for each position of the time series be denoted as $PE$. Then the elements of the matrix are:

$$ PE_{(pos,2i)} = \sin{(pos/10000^{2i/d_{model}})}$$
$$ PE_{(pos,2i+1)} = \cos{(pos/10000^{2i/d_{model}})}$$

where $pos$ - is the position, $i$ - index of the component of the corresponging vector, $d_{model}$ - dimension of each vector. Thus, even components represent sine values, and odd ones represent cosine values with different arguments.

***Task:*** propose ANOTHER way of positional encoding. Describe pros and cons of your method comparing to the described above.


The original Transformer implementation from the "Attention is All You Need" paper does not learn positional embeddings. Instead it uses a fixed static embedding.

Recently, Shaw et al. (2018) in his "Self-attention with relative position representations" used relative position
embedding (RPEs) with Transformers for machine translation. More recently, in Transformer pretrained language models, BERT (Devlin et al., 2018; Liu et al., 2019) used learnable PEs ( "Bert: Pre-training of deep bidirectional transformers for language understanding"). Yang et al. (2019) in "Generalized autoregressive pretraining for language understanding" modified RPEs and used them in the XLNet pre-trained
language model.

#### Fixed vs. learnable:

Fixed PEs are faster and more stable than letting a model learn these parameters itself.

However, one advantage that learnable positional encodings hold over fixed ones, is the flexibility to deal with non-sequential insertions into the data. For example, consider the BERT training objective. We set aside special tokens [CLS] and [SEP], to denote the sentence meaning and separation respectively. The latter have a well-defined positional meaning, but the former do not. A learned positional encoding can deal with this by ‘untying’ the [CLS] token. Ke et. al. treat this particular case by moving the [CLS] token to the zeroth sequential slot, and giving it its own shared parameter theta. This parameter is the same for all other tokens, indicating that the [CLS] token is equally “far” from all other tokens. Otherwise, there would be a bias to learn to give early sequence tokens more weight to [CLS].

##2. Practical part

###2.1 Basic algorithm

In the  https://github.com/skoltech-nlp/filimdb_evaluation  there is a dataset and the baseline -- translit\_baseline.py. The basic algorithm is based on the following idea: for transliteration, alphabetic n-grams from one language can be transformed into another language into n-grams of the same size, using the most frequent transformation rule found according to statistics on the training sample. To test the implementation, clone the repository, unzip the datasets, and run the evaluation script. To do this, you need to run the following commands:
* `git clone https://github.com/skoltech-nlp/filimdb_evaluation.git`
* `cd filimdb_evaluation`
* `./init.sh translit`
* `python evaluate_translit.py`


In [ ]:
!git clone https://github.com/skoltech-nlp/filimdb_evaluation.git

fatal: destination path 'filimdb_evaluation' already exists and is not an empty directory.


In [ ]:
!cd filimdb_evaluation

In [ ]:
!./init.sh translit

/bin/bash: ./init.sh: No such file or directory


In [ ]:
!!wget https://github.com/skoltech-nlp/filimdb_evaluation/blob/master/TRANSLIT.tar.gz

['--2021-05-13 23:20:50--  https://github.com/skoltech-nlp/filimdb_evaluation/blob/master/TRANSLIT.tar.gz',
 'Resolving github.com (github.com)... 140.82.114.3',
 'Connecting to github.com (github.com)|140.82.114.3|:443... connected.',
 'HTTP request sent, awaiting response... 200 OK',
 'Length: unspecified [text/html]',
 'Saving to: ‘TRANSLIT.tar.gz.1’',
 '',
 '',
 'TRANSLIT.tar.gz.1       [<=>                 ]       0  --.-KB/s               ',
 'TRANSLIT.tar.gz.1       [ <=>                ] 108.96K  --.-KB/s    in 0.04s   ',
 '',
 '2021-05-13 23:20:50 (2.79 MB/s) - ‘TRANSLIT.tar.gz.1’ saved [111577]',
 '']

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
from pathlib import Path
if IN_COLAB:
    google.colab.drive.mount("/content/drive")
    
    # Change this if you created the shortcut in a different location
    AUX_DATA_ROOT = Path("/content/drive/My Drive/HW4")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###2.2  Transformer-based approach


To implement your algorithm, use the template code, which needs to be modified.

Files to be used:
* `translit.py` - the main file where you need to make changes to be uploaded.
* `test_implementation.py` - file for testing the modules that you will need to implement as part of this task.
* `translit_utils` - folder with two `.py` files, which implements the logic of working with data and counting metrics.
First, you need to add some details in the code of the Transformer architecture, implement the methods of the class `LrScheduler`, which is responsible for updating the learning rate during training.
Next, you need to select the hyperparameters for the model according to the proposed guide.


Script `translit.py` contains the following elements:

* architecture of the Transformer as a class inheriting `torch.nn.Module`;
* algorithm of learning and classification in the form of  functions train and classify. Two these functions are the most important in this script.


####2.2.1  Positional Encoding

As you remember, Transformer treats an input sequence of elements as a time series. Since the Encoder inside the Transformer simultaneously processes the entire input sequence, the information about the position of the element needs to be encoded inside its embedding, since it is not identified in any other way inside the model. That is why the PositionalEncoding layer is used, which sums embeddings with a vector of the same dimension.
Let the matrix of these vectors for each position of the time series be denoted as $PE$. Then the elements of the matrix are:

$$ PE_{(pos,2i)} = \sin{(pos/10000^{2i/d_{model}})}$$
$$ PE_{(pos,2i+1)} = \cos{(pos/10000^{2i/d_{model}})}$$

where $pos$ - is the position, $i$ - index of the component of the corresponging vector, $d_{model}$ - dimension of each vector. Thus, even components represent sine values, and odd ones represent cosine values with different arguments.

In this task you are required to implement these formulas inside the class constructor *PositionalEncoding* in the main file ``translit.py``, which you are to upload. To run the test use the following command:

`python test_implementation.py test_positional_encoding`

Make sure that there is no any `AssertionError`!


####2.2.2  MultiHeadAttention


Then you are required to implement `attention` method in the class  `MultiHeadAttention`. The MultiHeadAttention layer takes as input  query vectors, key and value vectors for each step of the sequence of matrices  Q,K,V correspondingly. Each key vector, value vector, and query vector is obtained as a result of linear projection using one of three trained vector parameter matrices from the previous layer. This semantics can be represented in the form of formulas:
$$
Attention(Q, K, V)=softmax\left(\frac{Q K^{T}}{\sqrt{d_{k}}}\right) V\\
$$

$$
MultiHead(Q, K, V) = Concat\left(head_1, ... , head_h\right) W^O\\
$$

$$
head_i=Attention\left(Q W_i^Q, K W_i^K, V W_i^V\right)\\
$$
$h$ - the number of attention heads - parallel sub-layers for Scaled Dot-Product Attention on a vector of smaller dimension ($d_{k} = d_{q} = d_{v} = d_{model} / h$). 
The logic of  \texttt{MultiHeadAttention} is presented in the picture (from original  [paper](https://arxiv.org/abs/1706.03762)):

![](https://lilianweng.github.io/lil-log/assets/images/transformer.png)


Inside a method `attention` you are required to create a dropout layer from  MultiHeadAttention class constructor. Dropout layer is to be applied directly on the attention weights - the result of softmax operation. Value of drop probability  can be regulated in the train in the `model_config['dropout']['attention']`.

The correctness of implementation can be checked with
`test_implementation.py`:

```
python test_implementation.py test_multi_head_attention
```


####2.2.3  LrScheduler

The last thing you have to prepare is the class  `LrScheduler`, which is in charge of  learning rate updating after every step of the optimizer. You are required to fill the class constructor and the method `learning_rate`. The preferable stratagy of updating the learning rate (lr), is the following two stages:

* "warmup" stage - lr linearly increases until the defined value during the fixed number of steps (the proportion of all training steps - the parameter `train_config['warmup\_steps\_part']` in the train function). 
* "decrease" stage - lr linearly decreases until 0 during the left training steps.

`learning_rate()` call should return the value of  lr at this step,  which number is stored at self.step. The class constructor takes not only `warmup_steps_part` but the peak learning rate value `lr_peak` at the end of "warmup" stage and a string name of the strategy of learning rate scheduling. You can test other strategies if you want to with `self.type attribute`. 

Correctness check:

```
python test_implementation.py test_lr_scheduler
```


###2.3  Hyper-parameters choice

The model is ready. Now we need to find the optimal hyper-parameters.

In the repository in the TRANSLIT directory, in addition to train, there is also a dev samples and reduced versions of train and dev. The quality of models with different hyperparameters should be monitored on dev or on dev_small samples (in order to save time, since generating transliterations is a rather time-consuming process, comparable to one training epoch). Copy the existing code from `translit.py` into a new script and implement the required evaluation code there in the train function.
To generate predictions, you can use the `generate_predictions` function, to calculate the accuracy@1 metric, you can use the `compute_metrics` function from the `metrics.py script in the translit_utils directory.



Hyper-parameters are stored in the dictionary `model_config` and `train_config` in train function. The following hyperparameters in `model_config` and `train_config` are suggested to leave unmodified:

* n_layers $=$ 2
* n_heads $=$ 2
* hidden_size $=$ 128
* fc_hidden_size $=$ 256
* warmup_steps_part $=$ 0.1
* batch_size $=$ 200

 You can vary the dropout value. The model has 4 types of : ***embedding dropout*** applied on embdeddings before sending to the first layer of  Encoder or Decoder, ***attention*** dropout applied on the attention weights in the MultiHeadAttention layer, ***residual dropout*** applied on the output of each sublayer (MultiHeadAttention or FeedForward) in layers Encoder and Decoder and, finaly, ***relu dropout*** in used in FeedForward layer. For all 4 types it is suggested to test the same value of dropout from the list: 0.1, 0.15, 0.2.
 Also it is suggested to test several peak levels of learning rate - **lr_peak** : 5e-4, 1e-3, 2e-3.

Note that if you are using a GPU, then training one epoch takes about 1 minute, and up to 1 GB of video memory is required. When using the CPU, the learning speed slows down by about 2 times. If there are problems with insufficient RAM / video memory, reduce the batch size, but in this case the optimal range of learning rate values will change, and it must be determined again. To train a model with  batch_size $=$ 200 , it will take at least 300 epochs to achieve accuracy 0.66 on dev_small dataset.

*Question: What are the optimal hyperpameters according to your experiments? Add plots or other descriptions here.* 

```

ENTER HERE YOUR ANSWER

```



##3. Research part

We suggest to implement an additional regularization method - **label smoothing**. Now imagine that we have a prediction vector from probabilities at position t in the sequence of tokens for each token id from the vocabulary. CrossEntropy compares it with ground truth one-hot representation

$$[0, ... 0, 1, 0, ..., 0].$$

And now imagine that we are slightly "smoothed" the values in the ground truth vector and obtained

$$[\frac{\alpha}{|V|}, ..., \frac{\alpha}{|V|}, 1(1-\alpha)+\frac{\alpha}{|V|},  \frac{\alpha}{|V|}, ... \frac{\alpha}{|V|}],$$

where $\alpha$ - parameter from 0 to 1, $|V|$ - vocabulary size - number of components in the ground truth vector. The values ​​of this new vector are still summed to 1. Calculate the cross-entropy of our prediction vector and the new ground truth. Now, firstly, cross-entropy will never reach 0, and secondly, the result of the error function will require the model, as usual, to return the highest probability vector compared to other components of the probability vector for the correct token in the dictionary, but at the same time not too large, because as the value of this probability approaches 1, the value of the error function increases. For research on the use of label smoothing, see the [paper](https://arxiv.org/abs/1906.02629).
    
Accordingly, in order to embed label smoothing into the model, it is necessary to carry out the transformation described above on the ground truth vectors, as well as to implement the cross-entropy calculation, since the used `torch.nn.CrossEntropy` class is not quite suitable, since for the ground truth representation of `__call__` method takes the id of the correct token and builds a one-hot vector already inside. However, it is possible to implement what is required based on the internal implementation of this class [CrossEntropyLoss](https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html#CrossEntropyLoss).
    

Test different values of $\alpha$ (e.x, 0.05, 0.1, 0.2). Describe your experiments and results.


```

ENTER HERE YOUR ANSWER

```